In [1]:
import os
import numpy as np
import pandas as pd
import json

Fetching the Json Files


# Data Extraction

In [2]:
import os
# os.listdir('/content/drive/My Drive/SCIO_Final/SCIO/json')



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path_to_json = '/content/drive/My Drive/SCIO_Final/SCIO/json'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
print(len(json_files))  # for me this prints ['foo.json']


2988


In [5]:
votes_df = pd.DataFrame()
extracted_data = []
for index, js in enumerate(json_files[:50]):
  with open(os.path.join(path_to_json, js)) as json_file:
      json_text = json.load(json_file)
      extracted_data.append(json_text)
      votes_df = votes_df.append(json_text.get('scrutin'),ignore_index=True)


# Data Cleaning / Manipulation

In [ ]:
id_code = []
counter = 0
for i in votes_df['ventilationVotes']:
  for x in i.get('organe').get('groupes').get('groupe'):
    ref_value = x.get('vote').get('decompteNominatif').get('pours').get('votant') if x.get('vote').get('decompteNominatif').get('pours') is not None else False
    if isinstance(ref_value,list):
      id_code.append([str(det)[2:] for det in [k.get('acteurRef') for k in ref_value if k.get('acteurRef')] if det.startswith('PA')])
  counter += 1
votes_df['actuer'] = pd.Series(id_code) 


Manipluating the value for matching and chnaging the data types

Extracting the data from the Nodeputes file

In [8]:
nodeputes_df = pd.read_csv('/content/drive/My Drive/SCIO_Votes/nosdeputes_fr.csv',sep=';')
print(nodeputes_df.head())



    id                   nom  ... questions_ecrites questions_orales,,
0   32           Damien Abad  ...                37                8,,
1   43       Caroline Abadie  ...                 3                0,,
2  493  Jean-Félix Acquaviva  ...                22                2,,
3  152          Lénaïck Adam  ...                 1                0,,
4  234           Damien Adam  ...                 8                2,,

[5 rows x 40 columns]


Replacing the name to merge both files

In [38]:
# Rename the column values
nodeputes_df.rename(columns={'acteur':'actuer','questions_orales,,':'questions_orales'},inplace=True)
nodeputes_df.head()

,id,nom,nom_de_famille,prenom,sexe,date_naissance,lieu_naissance,num_deptmt,nom_circo,num_circo,mandat_debut,mandat_fin,ancien_depute,groupe_sigle,parti_ratt_financier,sites_web,emails,anciens_mandats,profession,place_en_hemicycle,url_an,id_an,slug,url_nosdeputes,url_nosdeputes_api,nb_mandats,twitter,semaines_presence,commission_presences,commission_interventions,hemicycle_interventions,hemicycle_interventions_courtes,amendements_proposes,amendements_signes,amendements_adoptes,rapports,propositions_ecrites,propositions_signees,questions_ecrites,questions_orales
0,32,Damien Abad,Abad,Damien,H,1980-04-05,Nîmes (Gard),01,Ain,5,2017-06-21,NaN,0,LR,Les Républicains,http://www.damienabad.fr|https://twitter.com/d...,NaN,NaN,0,85,http://www2.assemblee-nationale.fr/deputes/fic...,605036,damien-abad,https://www.nosdeputes.fr/damien-abad,https://www.nosdeputes.fr/damien-abad/csv,1,damienabad,40,49,22,306,349,134,3012,105,0,0,38,37,"8,,"
1,43,Caroline Abadie,Abadie,Caroline,F,1976-09-07,Saint-Martin-d'Hères (Isère),38,Isère,8,2017-06-21,NaN,0,LREM,La République en Marche,https://twitter.com/AbadieCaroline|https://fr-...,NaN,NaN,Industriel-Chef entreprise,200,http://www2.assemblee-nationale.fr/deputes/fic...,719866,caroline-abadie,https://www.nosdeputes.fr/caroline-abadie,https://www.nosdeputes.fr/caroline-abadie/csv,1,AbadieCaroline,33,66,18,21,8,14,335,189,0,0,4,3,"0,,"
2,493,Jean-Félix Acquaviva,Acquaviva,Jean-Félix,H,1973-03-19,Bastia (Haute-Corse),2B,Haute-Corse,2,2017-06-21,NaN,0,LT,Régions et peuples solidaires,https://fr-fr.facebook.com/jeanfelixacquaviva/...,NaN,NaN,Cadre,650,http://www2.assemblee-nationale.fr/deputes/fic...,719146,jean-felix-acquaviva,https://www.nosdeputes.fr/jean-felix-acquaviva,https://www.nosdeputes.fr/jean-felix-acquaviva...,1,JF_Acquaviva,25,33,48,121,11,188,2444,91,0,0,7,22,"2,,"
3,152,Lénaïck Adam,Adam,Lénaïck,H,1992-02-19,Saint Laurent du Maroni (Guyane),973,Guyane,2,2018-03-12,NaN,0,LREM,La République en Marche,https://fr-fr.facebook.com/lenaick.adam/|https...,NaN,NaN,Cadre supérieur (entreprises privée),357,http://www2.assemblee-nationale.fr/deputes/fic...,721036,lenaick-adam,https://www.nosdeputes.fr/lenaick-adam,https://www.nosdeputes.fr/lenaick-adam/csv,1,LenaickADAM,10,4,5,3,0,9,516,171,1,0,5,1,"0,,"
4,234,Damien Adam,Adam,Damien,H,1989-06-28,Orléans (Loiret),76,Seine-Maritime,1,2017-06-21,NaN,0,LREM,La République en Marche,https://twitter.com/damienadam76|http://damien...,NaN,NaN,Cadre supérieur (secteur privé),319,http://www2.assemblee-nationale.fr/deputes/fic...,722038,damien-adam,https://www.nosdeputes.fr/damien-adam,https://www.nosdeputes.fr/damien-adam/csv,1,damienadam76,29,93,167,15,4,69,663,222,0,0,5,8,"2,,"


Merging the files and converting it into CSV

In [39]:
#  Concatenating two dataframes
dat_df = pd.concat([votes_df,nodeputes_df],axis=1)
# data_df = pd.merge(votes_df, nodeputes_df, on="actuer")
print(dat_df.head())


                                              @xmlns  ... questions_orales
0  http://schemas.assemblee-nationale.fr/referentiel  ...              8,,
1  http://schemas.assemblee-nationale.fr/referentiel  ...              0,,
2  http://schemas.assemblee-nationale.fr/referentiel  ...              2,,
3  http://schemas.assemblee-nationale.fr/referentiel  ...              0,,
4  http://schemas.assemblee-nationale.fr/referentiel  ...              2,,

[5 rows x 60 columns]


In [40]:
dat_df.to_csv('Scio_Matched.csv')

# Sentiment Analysis

In [41]:

!pip install -q transformers


In [42]:
from __future__ import print_function
import ipywidgets as widgets
from transformers import pipeline

In [43]:
import pandas as pd

In [44]:
df = pd.read_csv("/content/Scio_Matched.csv", encoding='cp1252', index_col=0)

In [45]:
df.head()

,@xmlns,@xmlns:xsi,dateScrutin,demandeur,legislature,miseAuPoint,modePublicationDesVotes,numero,objet,organeRef,quantiemeJourSeance,seanceRef,sessionRef,sort,syntheseVote,titre,typeVote,uid,ventilationVotes,actuer,id,nom,nom_de_famille,prenom,sexe,date_naissance,lieu_naissance,num_deptmt,nom_circo,num_circo,mandat_debut,mandat_fin,ancien_depute,groupe_sigle,parti_ratt_financier,sites_web,emails,anciens_mandats,profession,place_en_hemicycle,url_an,id_an,slug,url_nosdeputes,url_nosdeputes_api,nb_mandats,twitter,semaines_presence,commission_presences,commission_interventions,hemicycle_interventions,hemicycle_interventions_courtes,amendements_proposes,amendements_signes,amendements_adoptes,rapports,propositions_ecrites,propositions_signees,questions_ecrites,questions_orales
0,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,2017-07-26,"{'texte': 'PrÃ©sident du groupe ""de la Gauche ...",15.0,"{'nonVotants': [None, None], 'pours': {'votant...",DecompteNominatif,76.0,"{'libelle': ""l'amendement nÂ° 269 de M. Rousse...",PO717460,2.0,RUANR5L15S2017IDS20630,SCR5A2017E1,"{'code': 'rejetÃ©', 'libelle': ""L'AssemblÃ©e n...","{'nombreVotants': '161', 'suffragesExprimes': ...",l'amendement nÂ° 269 de M. Roussel et les amen...,"{'codeTypeVote': 'SPO', 'libelleTypeVote': 'sc...",VTANR5L15V76,"{'organe': {'organeRef': 'PO717460', 'groupes'...","['609520', '605518', '719520', '719756', '7205...",32,Damien Abad,Abad,Damien,H,1980-04-05,NÃ®mes (Gard),01,Ain,5,2017-06-21,NaN,0,LR,Les RÃ©publicains,http://www.damienabad.fr|https://twitter.com/d...,NaN,NaN,0,85,http://www2.assemblee-nationale.fr/deputes/fic...,605036,damien-abad,https://www.nosdeputes.fr/damien-abad,https://www.nosdeputes.fr/damien-abad/csv,1,damienabad,40,49,22,306,349,134,3012,105,0,0,38,37,"8,,"
1,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,2018-04-10,"{'texte': 'PrÃ©sident du groupe ""de la Gauche ...",15.0,NaN,DecompteNominatif,464.0,"{'libelle': ""l'article premier du projet de lo...",PO717460,1.0,RUANR5L15S2018IDS20991,SCR5A2018O1,"{'code': 'adoptÃ©', 'libelle': ""l'AssemblÃ©e n...","{'nombreVotants': '88', 'suffragesExprimes': '...",l'article premier du projet de loi pour un nou...,"{'codeTypeVote': 'SPO', 'libelleTypeVote': 'sc...",VTANR5L15V464,"{'organe': {'organeRef': 'PO717460', 'groupes'...","['606712', '718850', '266776', '1012', '266797...",43,Caroline Abadie,Abadie,Caroline,F,1976-09-07,Saint-Martin-d'HÃ¨res (IsÃ¨re),38,IsÃ¨re,8,2017-06-21,NaN,0,LREM,La RÃ©publique en Marche,https://twitter.com/AbadieCaroline|https://fr-...,NaN,NaN,Industriel-Chef entreprise,200,http://www2.assemblee-nationale.fr/deputes/fic...,719866,caroline-abadie,https://www.nosdeputes.fr/caroline-abadie,https://www.nosdeputes.fr/caroline-abadie/csv,1,AbadieCaroline,33,66,18,21,8,14,335,189,0,0,4,3,"0,,"
2,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,2020-01-29,"{'texte': 'PrÃ©sidente du groupe ""Socialistes ...",15.0,"{'nonVotants': [None, None], 'pours': None, 'a...",DecompteNominatif,2424.0,"{'libelle': ""l'amendement de suppression nÂ° 1...",PO717460,1.0,RUANR5L15S2020IDS22043,SCR5A2020O1,"{'code': 'rejetÃ©', 'libelle': ""L'AssemblÃ©e n...","{'nombreVotants': '97', 'suffragesExprimes': '...",l'amendement de suppression nÂ° 140 de M. Saul...,"{'codeTypeVote': 'SPO', 'libelleTypeVote': 'sc...",VTANR5L15V2424,"{'organe': {'organeRef': 'PO717460', 'groupes'...","['721734', '719640', '720002', '720720', '7207...",493,Jean-FÃ©lix Acquaviva,Acquaviva,Jean-FÃ©lix,H,1973-03-19,Bastia (Haute-Corse),2B,Haute-Corse,2,2017-06-21,NaN,0,LT,RÃ©gions et peuples solidaires,https://fr-fr.facebook.com/jeanfelixacquaviva/...,NaN,NaN,Cadre,650,http://www2.assemblee-nationale.fr/deputes/fic...,719146,jean-felix-acquaviva,https://www.nosdeputes.fr/jean-felix-acquaviva,https://www.nosdeputes.fr/jean-felix-acquaviva...,1,JF_Acquaviva,25,33,48,121,11,188,2444,91,0,0,7,22,"2,,"
3,htt

In [46]:
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [47]:
df.head()

,@xmlns,@xmlns:xsi,dateScrutin,demandeur,legislature,miseAuPoint,modePublicationDesVotes,numero,objet,organeRef,quantiemeJourSeance,seanceRef,sessionRef,sort,syntheseVote,titre,typeVote,uid,ventilationVotes,actuer,id,nom,nom_de_famille,prenom,sexe,date_naissance,lieu_naissance,num_deptmt,nom_circo,num_circo,mandat_debut,mandat_fin,ancien_depute,groupe_sigle,parti_ratt_financier,sites_web,emails,anciens_mandats,profession,place_en_hemicycle,url_an,id_an,slug,url_nosdeputes,url_nosdeputes_api,nb_mandats,twitter,semaines_presence,commission_presences,commission_interventions,hemicycle_interventions,hemicycle_interventions_courtes,amendements_proposes,amendements_signes,amendements_adoptes,rapports,propositions_ecrites,propositions_signees,questions_ecrites,questions_orales
0,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,2017-07-26,"{'texte': 'PrÃ©sident du groupe ""de la Gauche ...",15.0,"{'nonVotants': [None, None], 'pours': {'votant...",DecompteNominatif,76.0,"{'libelle': ""l'amendement nÂ° 269 de M. Rousse...",PO717460,2.0,RUANR5L15S2017IDS20630,SCR5A2017E1,"{'code': 'rejetÃ©', 'libelle': ""L'AssemblÃ©e n...","{'nombreVotants': '161', 'suffragesExprimes': ...",l'amendement nÂ° 269 de M. Roussel et les amen...,"{'codeTypeVote': 'SPO', 'libelleTypeVote': 'sc...",VTANR5L15V76,"{'organe': {'organeRef': 'PO717460', 'groupes'...","['609520', '605518', '719520', '719756', '7205...",32,Damien Abad,Abad,Damien,H,1980-04-05,NÃ®mes (Gard),01,Ain,5,2017-06-21,NaN,0,LR,Les RÃ©publicains,http://www.damienabad.fr|https://twitter.com/d...,NaN,NaN,0,85,http://www2.assemblee-nationale.fr/deputes/fic...,605036,damien-abad,https://www.nosdeputes.fr/damien-abad,https://www.nosdeputes.fr/damien-abad/csv,1,damienabad,40,49,22,306,349,134,3012,105,0,0,38,37,"8,,"
1,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,2018-04-10,"{'texte': 'PrÃ©sident du groupe ""de la Gauche ...",15.0,NaN,DecompteNominatif,464.0,"{'libelle': ""l'article premier du projet de lo...",PO717460,1.0,RUANR5L15S2018IDS20991,SCR5A2018O1,"{'code': 'adoptÃ©', 'libelle': ""l'AssemblÃ©e n...","{'nombreVotants': '88', 'suffragesExprimes': '...",l'article premier du projet de loi pour un nou...,"{'codeTypeVote': 'SPO', 'libelleTypeVote': 'sc...",VTANR5L15V464,"{'organe': {'organeRef': 'PO717460', 'groupes'...","['606712', '718850', '266776', '1012', '266797...",43,Caroline Abadie,Abadie,Caroline,F,1976-09-07,Saint-Martin-d'HÃ¨res (IsÃ¨re),38,IsÃ¨re,8,2017-06-21,NaN,0,LREM,La RÃ©publique en Marche,https://twitter.com/AbadieCaroline|https://fr-...,NaN,NaN,Industriel-Chef entreprise,200,http://www2.assemblee-nationale.fr/deputes/fic...,719866,caroline-abadie,https://www.nosdeputes.fr/caroline-abadie,https://www.nosdeputes.fr/caroline-abadie/csv,1,AbadieCaroline,33,66,18,21,8,14,335,189,0,0,4,3,"0,,"
2,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,2020-01-29,"{'texte': 'PrÃ©sidente du groupe ""Socialistes ...",15.0,"{'nonVotants': [None, None], 'pours': None, 'a...",DecompteNominatif,2424.0,"{'libelle': ""l'amendement de suppression nÂ° 1...",PO717460,1.0,RUANR5L15S2020IDS22043,SCR5A2020O1,"{'code': 'rejetÃ©', 'libelle': ""L'AssemblÃ©e n...","{'nombreVotants': '97', 'suffragesExprimes': '...",l'amendement de suppression nÂ° 140 de M. Saul...,"{'codeTypeVote': 'SPO', 'libelleTypeVote': 'sc...",VTANR5L15V2424,"{'organe': {'organeRef': 'PO717460', 'groupes'...","['721734', '719640', '720002', '720720', '7207...",493,Jean-FÃ©lix Acquaviva,Acquaviva,Jean-FÃ©lix,H,1973-03-19,Bastia (Haute-Corse),2B,Haute-Corse,2,2017-06-21,NaN,0,LT,RÃ©gions et peuples solidaires,https://fr-fr.facebook.com/jeanfelixacquaviva/...,NaN,NaN,Cadre,650,http://www2.assemblee-nationale.fr/deputes/fic...,719146,jean-felix-acquaviva,https://www.nosdeputes.fr/jean-felix-acquaviva,https://www.nosdeputes.fr/jean-felix-acquaviva...,1,JF_Acquaviva,25,33,48,121,11,188,2444,91,0,0,7,22,"2,,"
3,htt

In [ ]:
import re
import json
from collections import Counter
nlp_sentence_classif = pipeline('sentiment-analysis')
scores = []
for i in range(0,len(df)):
  try:
    if re.findall('^{',df['syntheseVote'][i]):
      w = eval(df['syntheseVote'][i])
      scoring = nlp_sentence_classif(w.get('annonce'))
      scores.append([dict(Counter(["POS" if text.get('label') == "POSITIVE" else "NEG" for text in scoring])),w.get('annonce')])
  except TypeError as e :
    print(df['syntheseVote'].iloc[i])
print(scores)
    

In [49]:
def statement_polarity(pos , neg):
  if pos :
    return "POSITIVE STATEMENT"
  elif pos == neg:
    return "NEGATIVE STATEMENT"
  elif pos > neg:
    return "POSITIVE STATEMENT"
  elif neg:
    return "NEGATIVE STATEMENT"


In [50]:
df['statement_type'] = df.apply(lambda x: statement_polarity(x.get('POS') if x.get('POS') else 0, x.get('NEG') if x.get('NEG') else 0), axis=1)

In [51]:
df = df.loc[:, ~df.columns.duplicated()]
print(df.columns)
pos = []
neg = []
for sco in scores:
  if sco[0].get('POS'):
    pos.append(sco[0].get('POS'))
  elif sco[0].get('NEG'):
    neg.append(sco[0].get('NEG'))
print("ssssssssssssssssssssssssssssssssssssss", pos)
print("<PPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPP", neg)
# df['POS '] = pd.DataFrame(pos)
df['NEG ']  = pd.DataFrame(neg)
# print(df.head())
# print("XXXXXXXXXXX",df[df['POS '] == df['POS '].isnull()])
# print(df['NEG '])
# # df['POS'] = pd.DataFrame([sco[0].values()  if sco[0].get('POS') else  False  for sco in scores])
# # df['NEG'] = pd.DataFrame([sco[0].values() if sco[0].get('NEG') else False for sco in scores  ])
# if df['POS ']:
#   df['POS '] = pd.to_numeric(df['POS '], errors='coerce') 
#   df = df.dropna(subset=['POS '])
#   df['POS '] = df['POS '].astype(int) 
# if df['NEG ']:
df['NEG '] = pd.to_numeric(df['NEG '], errors='coerce')
df = df.dropna(subset=['NEG '])
df['NEG '] = df['NEG '].astype(int) 
df.head()

Index(['@xmlns', '@xmlns:xsi', 'dateScrutin', 'demandeur', 'legislature',
       'miseAuPoint', 'modePublicationDesVotes', 'numero', 'objet',
       'organeRef', 'quantiemeJourSeance', 'seanceRef', 'sessionRef', 'sort',
       'syntheseVote', 'titre', 'typeVote', 'uid', 'ventilationVotes',
       'actuer', 'id', 'nom', 'nom_de_famille', 'prenom', 'sexe',
       'date_naissance', 'lieu_naissance', 'num_deptmt', 'nom_circo',
       'num_circo', 'mandat_debut', 'mandat_fin', 'ancien_depute',
       'groupe_sigle', 'parti_ratt_financier', 'sites_web', 'emails',
       'anciens_mandats', 'profession', 'place_en_hemicycle', 'url_an',
       'id_an', 'slug', 'url_nosdeputes', 'url_nosdeputes_api', 'nb_mandats',
       'twitter', 'semaines_presence', 'commission_presences',
       'commission_interventions', 'hemicycle_interventions',
       'hemicycle_interventions_courtes', 'amendements_proposes',
       'amendements_signes', 'amendements_adoptes', 'rapports',
       'propositions_ecrites', 

,@xmlns,@xmlns:xsi,dateScrutin,demandeur,legislature,miseAuPoint,modePublicationDesVotes,numero,objet,organeRef,quantiemeJourSeance,seanceRef,sessionRef,sort,syntheseVote,titre,typeVote,uid,ventilationVotes,actuer,id,nom,nom_de_famille,prenom,sexe,date_naissance,lieu_naissance,num_deptmt,nom_circo,num_circo,mandat_debut,mandat_fin,ancien_depute,groupe_sigle,parti_ratt_financier,sites_web,emails,anciens_mandats,profession,place_en_hemicycle,url_an,id_an,slug,url_nosdeputes,url_nosdeputes_api,nb_mandats,twitter,semaines_presence,commission_presences,commission_interventions,hemicycle_interventions,hemicycle_interventions_courtes,amendements_proposes,amendements_signes,amendements_adoptes,rapports,propositions_ecrites,propositions_signees,questions_ecrites,questions_orales,statement_type,NEG
0,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,2017-07-26,"{'texte': 'PrÃ©sident du groupe ""de la Gauche ...",15.0,"{'nonVotants': [None, None], 'pours': {'votant...",DecompteNominatif,76.0,"{'libelle': ""l'amendement nÂ° 269 de M. Rousse...",PO717460,2.0,RUANR5L15S2017IDS20630,SCR5A2017E1,"{'code': 'rejetÃ©', 'libelle': ""L'AssemblÃ©e n...","{'nombreVotants': '161', 'suffragesExprimes': ...",l'amendement nÂ° 269 de M. Roussel et les amen...,"{'codeTypeVote': 'SPO', 'libelleTypeVote': 'sc...",VTANR5L15V76,"{'organe': {'organeRef': 'PO717460', 'groupes'...","['609520', '605518', '719520', '719756', '7205...",32,Damien Abad,Abad,Damien,H,1980-04-05,NÃ®mes (Gard),01,Ain,5,2017-06-21,NaN,0,LR,Les RÃ©publicains,http://www.damienabad.fr|https://twitter.com/d...,NaN,NaN,0,85,http://www2.assemblee-nationale.fr/deputes/fic...,605036,damien-abad,https://www.nosdeputes.fr/damien-abad,https://www.nosdeputes.fr/damien-abad/csv,1,damienabad,40,49,22,306,349,134,3012,105,0,0,38,37,"8,,",NEGATIVE STATEMENT,1
1,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,2018-04-10,"{'texte': 'PrÃ©sident du groupe ""de la Gauche ...",15.0,NaN,DecompteNominatif,464.0,"{'libelle': ""l'article premier du projet de lo...",PO717460,1.0,RUANR5L15S2018IDS20991,SCR5A2018O1,"{'code': 'adoptÃ©', 'libelle': ""l'AssemblÃ©e n...","{'nombreVotants': '88', 'suffragesExprimes': '...",l'article premier du projet de loi pour un nou...,"{'codeTypeVote': 'SPO', 'libelleTypeVote': 'sc...",VTANR5L15V464,"{'organe': {'organeRef': 'PO717460', 'groupes'...","['606712', '718850', '266776', '1012', '266797...",43,Caroline Abadie,Abadie,Caroline,F,1976-09-07,Saint-Martin-d'HÃ¨res (IsÃ¨re),38,IsÃ¨re,8,2017-06-21,NaN,0,LREM,La RÃ©publique en Marche,https://twitter.com/AbadieCaroline|https://fr-...,NaN,NaN,Industriel-Chef entreprise,200,http://www2.assemblee-nationale.fr/deputes/fic...,719866,caroline-abadie,https://www.nosdeputes.fr/caroline-abadie,https://www.nosdeputes.fr/caroline-abadie/csv,1,AbadieCaroline,33,66,18,21,8,14,335,189,0,0,4,3,"0,,",NEGATIVE STATEMENT,1
2,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,2020-01-29,"{'texte': 'PrÃ©sidente du groupe ""Socialistes ...",15.0,"{'nonVotants': [None, None], 'pours': None, 'a...",DecompteNominatif,2424.0,"{'libelle': ""l'amendement de suppression nÂ° 1...",PO717460,1.0,RUANR5L15S2020IDS22043,SCR5A2020O1,"{'code': 'rejetÃ©', 'libelle': ""L'AssemblÃ©e n...","{'nombreVotants': '97', 'suffragesExprimes': '...",l'amendement de suppression nÂ° 140 de M. Saul...,"{'codeTypeVote': 'SPO', 'libelleTypeVote': 'sc...",VTANR5L15V2424,"{'organe': {'organeRef': 'PO717460', 'groupes'...","['721734', '719640', '720002', '720720', '7207...",493,Jean-FÃ©lix Acquaviva,Acquaviva,Jean-FÃ©lix,H,1973-03-19,Bastia (Haute-Corse),2B,Haute-Corse,2,2017-06-21,NaN,0,LT,RÃ©gions et peuples solidaires,https://fr-fr.facebook.com/jeanfelixacquaviva/...,NaN,NaN,Cadre,650,http://www2.assemblee-nationale.fr/deputes/fic...,719146,jean-felix-acquaviva,https://www.nosdeputes.fr/jean-felix-acquaviva,https://www.nosdeputes.fr/jean-felix-acquaviva...,1,